In [1]:
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\python311\lib\site-packages (2.18.0)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [3]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [4]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize image pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift image width
    height_shift_range=0.2,  # Randomly shift image height
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Randomly flip the image
    fill_mode='nearest'  # Filling pixels when transformations are done
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 762 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


In [ ]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
%pip install scipy


Note: you may need to restart the kernel to use updated packages.Collecting scipy
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 660.6 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 667.8 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 667.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 323.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/43.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.9 MB 1.9 MB/s eta 0:00:23
   ---------------------------------------- 0.1/43.9 MB 1.7 MB/s eta 0:00:26
   ---------------------------------------- 0.1/43.9 MB 1.7 MB/s eta 0:00:26
   ---------------------------------------- 0.2/43.9 MB 913.1 kB/s eta 0:00:48
   ---------------------------------------- 0.3/43.9 MB 1.4 MB/s eta 0:00:33
   ---------------------------------------- 0.3/43.9 MB 1.4 MB/s eta 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.3011 - loss: 1.5877

c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 375ms/step - accuracy: 0.3015 - loss: 1.5815 - val_accuracy: 0.4271 - val_loss: 1.3460
Epoch 2/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 5s 235ms/step - accuracy: 0.3438 - loss: 1.3528

c:\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.3438 - loss: 1.3528 - val_accuracy: 0.4688 - val_loss: 1.3266
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 351ms/step - accuracy: 0.3449 - loss: 1.3504 - val_accuracy: 0.5208 - val_loss: 1.2121
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5000 - loss: 1.2195 - val_accuracy: 0.5104 - val_loss: 1.2378
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 350ms/step - accuracy: 0.3824 - loss: 1.2659 - val_accuracy: 0.5104 - val_loss: 1.2220
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4688 - loss: 1.0981 - val_accuracy: 0.5000 - val_loss: 1.2183
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 342ms/step - accuracy: 0.4592 - loss: 1.1483 - val_accuracy: 0.5208 - val_loss: 1.1773
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5000 - loss: 1.1447 - val_accuracy: 0.5000 - val_loss: 1.2211
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 351ms/step - accuracy: 0.4582 - loss: 1.1638 - val_accuracy: 0.5417 - val_loss

In [11]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.4914 - loss: 1.2866
Test accuracy: 0.5252525210380554


In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data\test\download.jpeg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
The snake species is: Python molurus
